In [1]:
%matplotlib tk
import pandas as pd
from datetime import date
import seaborn as sns
import pandas as pd
import glob
import matplotlib.pyplot as plt

file_directory = ''
csv_files = glob.glob(file_directory + '*.csv')

data_frames = []
for file_path in csv_files:
    data = pd.read_csv(file_path)
    cleaned_data = data.dropna(axis=1, how='all').dropna(axis=0, how='all')
    cleaned_data.columns = [col.replace('.', '_') for col in cleaned_data.columns]
    data_frames.append(cleaned_data)

In [2]:
data_frames[0].head()

,text_started,key_resp_keys,key_resp_rt,key_resp_started,cond_text_started,Inst_text_started,key_resp_3_keys,key_resp_3_rt,key_resp_3_started,sound_1_started,...,T6Q10R_keys,T6Q10R_corr,T6Q10R_rt,T6Q10R_started,participant,session,date,expName,psychopyVersion,frameRate
0,14.157415,space,22.538189,14.157415,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,p1,1.0,2024_Mar_04_1133,TCSMF,2021.2.3,143.943298
1,NaN,NaN,NaN,NaN,37.104032,37.104032,space,7.335893,37.104032,0.005602,...,2.0,1.0,6.458582,1836.978653,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
summary_data = pd.DataFrame(columns=['Participant', 'Trial', 'Condition', 'Percentage Correct', 'Average Reaction Time'])
conditions = ['Clear_1.0x', 'Clear_1.5x', 'Clear_2.0x', 'Noise_1.0x', 'Noise_1.5x', 'Noise_2.0x']
# Initialize an empty DataFrame to hold the master data
master_data = pd.DataFrame()
# Loop over each DataFrame in the 'data_frame' object
for l in data_frames:
    summary_data = pd.DataFrame()
    for trial, condition in zip(range(1, 7), conditions):
        total_correct = sum(cleaned_data[f"""T{trial}Q{i}R_corr"""] for i in range(1, 11)).values[1]
        total_correct = (total_correct / 10) * 100
        avg_reaction_time = sum(cleaned_data[f'T{trial}Q{i}R_rt'] for i in range(1, 11)).values[1] / 10
        pnum = cleaned_data['participant'].values[0]
        summary_data = summary_data._append({'Participant': pnum, 'Trial': trial, 'Percentage Correct': total_correct,
                                            'Average Reaction Time': avg_reaction_time, 'Condition': condition}, ignore_index=True)
    # Append the summary data to the master DataFrame
    master_data = pd.concat([master_data, summary_data])

today = date.today()
# Save the master DataFrame to a CSV file
master_data.to_csv(f'TCSMF_master_{today}.csv', index=False)

In [4]:
summary_data.columns = ['P_Num', 'Trial', 'Cond', 'Corr', 'rt']
g_data = summary_data.groupby(['P_Num', 'Cond'])
g_data = g_data['Corr'].mean().reset_index()

In [5]:
mast_data = pd.read_csv('TCSMF_master_2024-04-30.csv')
grouped_data = mast_data.groupby(['P_Num', 'Cond'])
avg_correct = grouped_data['Corr'].mean().reset_index()
avg_correct['Corr'] = avg_correct['Corr'] * 100
final_data = pd.concat([avg_correct, g_data], axis=0)

FileNotFoundError: [Errno 2] No such file or directory: 'TCSMF_master_2024-04-30.csv'

: 

In [ ]:
colors = ['dimgray', 'darkgray', 'gainsboro', 'dimgray', 'darkgray', 'gainsboro']
plt.figure(figsize=(10, 6))
sns.barplot(x='Cond', y='Corr', data=final_data, errorbar='se', capsize=0.1, palette=colors, hue='Cond', legend=True, linewidth=1.5)
sns.stripplot(x='Cond', y='Corr', data=final_data, color='black', size=5, jitter=True)
plt.xlabel('Condition')
plt.ylabel('Average Percentage Correct (%)')
plt.title('Average Percentage Correct per Participant and Condition')
plt.show()